In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import gaussian_kde
here = globals()['_dh'][0]
plt.style.use(os.path.join(here,"../etc/config/geoff_stylesheet.mplstyle"))
import seaborn as sns
from sklearn.metrics import r2_score
import torch

import matplotlib as mpl
%matplotlib inline
mpl.rc_file_defaults()

In [2]:
basedir = "../results/cinic_example/"
ind_prob_paths = [
    os.path.join(basedir, basename)
    for basename in os.listdir(basedir)
    if "base_wideresnet28_10ind_preds" in basename
]
# basedir = "../models/cinic_wrn28_10/"
ood_prob_paths = [
    os.path.join(basedir, basename)
    for basename in os.listdir(basedir)
    if "base_wideresnet28_10ood_cinic_preds" in basename
]
num_classes = 10

In [3]:
ind_probs = torch.stack([
    torch.tensor(np.load(ind_prob_path)).float()
    for ind_prob_path in ind_prob_paths
], dim=-2).softmax(dim=-1)
ind_labels = torch.tensor(np.load(ind_prob_paths[0].replace("preds", "labels"))).long()
ind_indices = ind_labels
# ind_indices = torch.randperm(len(ind_probs))[:10000]

ood_probs = torch.stack([
    torch.tensor(np.load(ood_prob_path)).float()
    for ood_prob_path in ood_prob_paths
], dim=-2).softmax(dim=-1)
ood_labels = torch.tensor(np.load(ood_prob_paths[0].replace("preds", "labels"))).long()
ood_indices = ood_labels
# ood_indices = torch.randperm(len(ood_probs))[:10000]

In [4]:
ind_ef2 = ind_probs[ind_indices].pow(2).sum(dim=-1).mean(dim=-1)
ood_ef2 = ood_probs[ood_indices].pow(2).sum(dim=-1).mean(dim=-1)

In [5]:
ind_var = ind_probs[ind_indices].var(dim=-2).sum(dim=-1)
ood_var = ood_probs[ood_indices].var(dim=-2).sum(dim=-1)

In [6]:
ind_probs[ind_indices]

tensor([[[9.9985e-01, 8.1714e-06, 3.3780e-05,  ..., 5.3157e-06,
          3.8632e-05, 3.2646e-05],
         [9.9958e-01, 2.1336e-05, 1.0072e-04,  ..., 1.5654e-05,
          1.4999e-04, 6.4279e-05],
         [9.9970e-01, 1.5725e-05, 4.8781e-05,  ..., 8.6205e-06,
          1.5936e-04, 1.6809e-05],
         [9.9908e-01, 4.1518e-05, 1.3902e-04,  ..., 2.7399e-05,
          4.6640e-04, 1.2222e-04],
         [9.9951e-01, 3.6673e-05, 2.2757e-04,  ..., 1.4393e-05,
          9.0735e-05, 3.7059e-05]],

        [[1.7842e-05, 1.5482e-05, 1.1705e-05,  ..., 2.8837e-05,
          1.0219e-05, 9.6055e-06],
         [5.5938e-06, 5.9320e-06, 4.5542e-06,  ..., 8.2829e-06,
          3.4784e-06, 3.7468e-06],
         [1.4686e-05, 1.1770e-05, 1.4212e-05,  ..., 2.2668e-05,
          1.3522e-05, 1.0842e-05],
         [5.3980e-06, 5.7781e-06, 5.0772e-06,  ..., 1.0552e-05,
          3.7193e-06, 4.7021e-06],
         [5.9710e-06, 5.6661e-06, 7.6201e-06,  ..., 7.3081e-06,
          3.5808e-06, 4.8113e-06]],

      

In [7]:
ind_ef2_kde = gaussian_kde(ind_ef2, bw_method=len(ind_indices) ** (-.25))
ood_ef2_kde = gaussian_kde(ood_ef2, bw_method=len(ood_indices) ** (-.25))

In [8]:
ind_joint_kde = gaussian_kde(np.stack([ind_ef2, ind_var]), bw_method=len(ind_indices) ** (-.25))
ood_joint_kde = gaussian_kde(np.stack([ood_ef2, ood_var]), bw_method=len(ood_indices) ** (-.25))

In [9]:
xs = np.linspace(0., 1., 101)
x_grid, y_grid = np.meshgrid(xs, xs)
joint = np.stack([x_grid.reshape(-1), y_grid.reshape(-1)])

In [ ]:
import gpytorch

class GPModel(gpytorch.models.ExactGP):
    def __init__(self, metrics, variances):
        likelihood = gpytorch.likelihoods.GaussianLikelihood()
        super().__init__(metrics, variances, likelihood)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.RBFKernel()
        self.covar_module.initialize(lengthscale=(len(variances) ** (-.25)))
        self.likelihood.initialize(noise=1e-4)
        
    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)


ind_model = GPModel(ind_ef2.double(), ind_var.double()).double()
ood_model = GPModel(ood_ef2.double(), ood_var.double()).double()
ind_model.eval()
ood_model.eval()

start_conf = 1. / num_classes
cond_expec_xs = torch.linspace(start_conf, 1., int((1. - start_conf) * 100) + 1)
with torch.no_grad(), gpytorch.settings.skip_posterior_variances():
    with gpytorch.settings.max_cholesky_size(1e6):
        ind_cond_expec = ind_model(torch.tensor(cond_expec_xs).double()).mean
        ood_cond_expec = ood_model(torch.tensor(cond_expec_xs).double()).mean
        
        ind_preds_ind = ind_model(torch.tensor(ind_ef2.double())).mean
        ood_preds_ood = ood_model(torch.tensor(ood_ef2.double())).mean
        ind_preds_ood = ind_model(torch.tensor(ood_ef2.double())).mean
        ood_preds_ind = ood_model(torch.tensor(ind_ef2.double())).mean
        print("R^2 ind predicts ind: {}".format(r2_score(ind_var,ind_preds_ind)))
        print("R^2 ood predicts ind: {}".format(r2_score(ind_var,ood_preds_ind)))
        print("R^2 ind predicts ood: {}".format(r2_score(ood_var,ood_preds_ood)))
        print("R^2 ood predicts ood: {}".format(r2_score(ood_var,ind_preds_ood)))

/var/folders/1f/b2_sjx654d35kr09mh0stb5r0000gn/T/ipykernel_98343/1006032397.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ind_cond_expec = ind_model(torch.tensor(cond_expec_xs).double()).mean
/var/folders/1f/b2_sjx654d35kr09mh0stb5r0000gn/T/ipykernel_98343/1006032397.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ood_cond_expec = ood_model(torch.tensor(cond_expec_xs).double()).mean


In [ ]:
fig, (var_ax, ind_cond_ax, ood_cond_ax, cond_exp_ax) = plt.subplots(
    1, 4, figsize=(12, 3), sharex=True, sharey=False
)
levels = np.linspace(-3., 3., 51)

sns.kdeplot(ind_var, ax=var_ax)
sns.kdeplot(ood_var, ax=var_ax)
var_ax.set(xlabel="Var.", title="Marginal Var. Dist.\nComparison", ylim=(0., 15.))

ind_vals = ind_joint_kde(joint).reshape(x_grid.shape)
ind_vals = ind_vals / ind_ef2_kde(x_grid.ravel()).reshape(x_grid.shape)
ind_vals = np.where(x_grid < (1. / num_classes), 0., ind_vals)
f = ind_cond_ax.contourf(
    x_grid, y_grid, ind_vals.clip(0., 10.),
    cmap="Blues",
    levels=np.linspace(0., 10., 50),
)
ind_cond_ax.set(
    xlim=(0., 1.), ylim=(0., 1.), xlabel=r"Avg. Conf. ($ E[ \Vert f \Vert^2 ]$)",
    ylabel=r"Variance", title="Conditional Var. Dist.\nCIFAR10 (InD)"
)
fig.colorbar(f, ax=ind_cond_ax)

ood_vals = ood_joint_kde(joint).reshape(x_grid.shape)
ood_vals = ood_vals / ood_ef2_kde(x_grid.ravel()).reshape(x_grid.shape)
ood_vals = np.where(x_grid < (1. / num_classes), 0., ood_vals)
f = ood_cond_ax.contourf(
    x_grid, y_grid, ood_vals.clip(0., 10.),
    cmap="Oranges",
    levels=np.linspace(0., 10., 50),
)
ood_cond_ax.set(
    xlabel=r"Avg. Conf. ($ E[ \Vert f \Vert^2 ]$)",
    title="Conditional Var. Dist.\nCINIC10 (OOD)",
    yticks=[]
)
fig.colorbar(f, ax=ood_cond_ax)

cond_exp_ax.plot(cond_expec_xs, ind_cond_expec, label="CIFAR10 (InD)")
cond_exp_ax.plot(cond_expec_xs, ood_cond_expec, label="CINIC10 (OOD)")
cond_exp_ax.set(
    ylim=(0., 1.), 
    ylabel="Conditional Variance",#r"$E [ \textrm{Var} \mid \textrm{Conf} ]$",
    xlabel="Avg. Confidence ",#r"Avg. Conf. ($ E[ \Vert f \Vert^2 ]$)",
    title="Conditionally Expected Var.\nComparison"
)
cond_exp_ax.legend(loc="best")
fig.tight_layout()

In [ ]:
model_indices = torch.tensor([
    [0, 1, 2, 3],
    [1, 2, 3, 4],
    [2, 3, 4, 0],
    [3, 4, 0, 1],
    [4, 0, 1, 2]
])
bs_ind_vars = (ind_probs[ind_indices].permute(1, 0, 2)[model_indices].
               var(dim=-3).sum(dim=-1).mean(dim=-1))
bs_ood_vars = (ood_probs[ood_indices].permute(1, 0, 2)[model_indices].
               var(dim=-3).sum(dim=-1).mean(dim=-1))
bs_ind_vars, bs_ood_vars

In [ ]:
fig, (var_ax, cond_exp_ax) = plt.subplots(
    1, 2, figsize=(6, 3), sharex=False, sharey=False
)
print([ind_var.mean().item(), ood_var.mean().item()])
sns.barplot(x=["CIFAR10\n(InD)", "CINIC10\n(OOD)"], y=[ind_var.mean().item(), ood_var.mean().item()], ax=var_ax)
var_ax.set(ylabel="Avg. Var.", title="Marginal Var.\nComparison")

cond_exp_ax.plot(1. - cond_expec_xs, ind_cond_expec, label="CIFAR10 (InD)")
cond_exp_ax.plot(1. - cond_expec_xs, ood_cond_expec, label="CINIC10 (OOD)")
cond_exp_ax.set(
    ylim=(0., 0.5), xlim=(0., 1.),
    ylabel=r"$E [ \textrm{Var} \mid \textrm{Uncertainty} ]$",
    xlabel=r"Aleatoric Uncertainty",
    title="Conditionally Expected Var.\nComparison"
)
cond_exp_ax.legend(loc="best")
fig.tight_layout()
fig.subplots_adjust(wspace=0.5)
fig.savefig("conditional_fig_talk.pdf", transparent=True)

In [ ]:
# xs = torch.rand(20).mul(5.).sort()[0]
# model = GPModel(xs, torch.randn(xs.size(0)))
# model.eval()
# with torch.no_grad(), gpytorch.settings.prior_mode():
#     ys = model(xs).rsample()
# plt.plot(xs, ys)

In [ ]:
# model = GPModel(xs, ys)
# model.eval()

# ind_xs = torch.linspace(0., 5., 2000)
# ood_xs = torch.linspace(-5., 0., 2000)
# model.eval()
# with torch.no_grad():
#     preds = model(torch.cat([ood_xs, ind_xs]))
#     lower, upper = preds.confidence_region()
#     ind_var = model(ind_xs).variance
#     ood_var = model(ood_xs).variance
    
# plt.fill_between(torch.cat([ood_xs, ind_xs]), lower, upper, alpha=0.3)
# plt.plot(torch.cat([ood_xs, ind_xs]), preds.mean)

In [ ]:
# ind_noise = torch.sin(ind_xs * 4).pow(2.).mul(2.).add(1e-2)
# ood_noise = torch.sin(ood_xs * 4).pow(2.).mul(2.).add(1e-2)

In [ ]:
# ind_test_model = GPModel((ind_noise).double(), ind_var.double()).double()
# ood_test_model = GPModel((ood_noise).double(), ood_var.double()).double()
# ind_test_model.eval()
# ood_test_model.eval()

# cond_expec_xs = torch.linspace(0., 2., 101)
# with torch.no_grad(), gpytorch.settings.skip_posterior_variances():
#     with gpytorch.settings.max_cholesky_size(1e6):
#         ind_test_cond_expec = ind_test_model(torch.tensor(cond_expec_xs).double()).mean
#         ood_test_cond_expec = ood_test_model(torch.tensor(cond_expec_xs).double()).mean

In [ ]:
# plt.plot(cond_expec_xs, ind_test_cond_expec)
# plt.plot(cond_expec_xs, ood_test_cond_expec)

In [ ]:
# plt.scatter((ind_noise).double(), ind_var.double(), s=2)
# plt.scatter((ood_noise).double(), ood_var.double(), s=2)